<a href="https://colab.research.google.com/github/jcmachicao/cisti2025_earlysymbolicknowledge/blob/main/pp_symbolic_read_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade --force-reinstall gensim

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 4.6 MB/s eta 0:00:00
  Attempting uninstall: wrapt
    Found existing installation: wrapt 1.17.2
    Uninstalling wrapt-1.17.2:
      Successfully uninstalled wrapt-1.17.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
  Attempting uninstall: smart-open
    Found existing installation: smart-open 7.1.0
    Uninstalling smart-open-7.1.0:
      Successfully uninsta

# Experiment (Architecture A_6)

In [ ]:
import os, json
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
import datetime

In [ ]:
lab_data = pd.DataFrame()

In [ ]:
# Cargar frases desde los archivos JSON
# path = "D://2025 hot writing/pp_symbolic_knowledge_LLM/"
path = "drive/My Drive/00 2025_all/2025 Papers/"
os.listdir(path)

['pp_GovCxSys__simulation-framework.py',
 'pp_simul__complexsystem_governance.ipynb',
 'pp_simul__cognitive_evolution.ipynb',
 'general_simbolico.ipynb',
 'db__ventas_etica.json',
 'db__general_phrases_v4.json',
 'db__ventas_situaciones.json',
 'pp_symbolic_read.ipynb',
 'db__spatial_phrases.json',
 'results.xlsx']

In [ ]:
# Experimento A
palabras_objetivo = ["corrupción", "riesgo", "ética", "ilegal", "engaño", "fraude", "manipulación", "regalo", "presión", "soborno"]
archivo_base = "db__ventas_situaciones.json"
archivo_condiciones = "db__ventas_etica.json"
# 🧪 Probar con una frase
frases = [
    "El vendedor da un regalo al gerente de la empresa",
    "La vendedora tiene intenciones de no transmitir toda la información al cliente",
    "El apuro hace que no se consulten los detalles del producto antes de la venta"
  ]

In [ ]:
# Experimento B
palabras_objetivo = ["caida", "ruptura", "resbalar", "tritura", "impacto", "desorden", "riesgo", "quebrado", "roto", "ruptura"]
archivo_base = "db__general_phrases_v4.json"
archivo_condiciones = "db__spatial_phrases.json"
# 🧪 Probar con una frase
frases = [
    "El plato está al borde de la mesa",
    "Las tazas están siendo manipuladas sin cuidado",
    "Hay muchos platos superpuestos uno sobre otro sin equilibrio"
    ]

In [ ]:
frases_total = [
    "El vendedor da un regalo al gerente de la empresa",
    "La vendedora tiene intenciones de no transmitir toda la información al cliente",
    "El apuro hace que no se consulten los detalles del producto antes de la venta",
    "El plato está al borde de la mesa",
    "Las tazas están siendo manipuladas sin cuidado",
    "Hay muchos platos superpuestos uno sobre otro sin equilibrio"
    ]

In [ ]:
frases_df = pd.DataFrame(frases_total, columns=['frases'])
frases_df.to_excel(path+"frases_test.xlsx")

In [ ]:
# 📥 Función para cargar frases
def cargar_sentencias(path, archivo, clave):
    with open(os.path.join(path, archivo), "r", encoding="utf-8") as f:
        data = json.load(f)
    return [s.split() for s in data[clave]]

# 🧹 Si querés limpieza básica:
def limpiar_frase(frase):
    return frase.lower().replace(".", "").replace(",", "").replace("¿", "").replace("?", "").replace("¡", "").replace("!", "")

# ✅ Cargar frases base y condicionales
general_sentences = cargar_sentencias(path, archivo_base, "phrases")
condition_sentences = cargar_sentencias(path, archivo_condiciones, "phrases")

In [ ]:
len(general_sentences), len(condition_sentences)

(305, 73)

In [ ]:
# 🧠 Entrenar modelos Word2Vec
def entrenar_w2v(sentences, vector_size=50, window=5, min_count=1, sg=1, epochs=200):
    return Word2Vec(
        sentences=sentences,
        vector_size=vector_size,
        window=window,
        min_count=min_count,
        workers=4,
        sg=sg,
        epochs=epochs
    )

w2v_base = entrenar_w2v(general_sentences)
w2v_combined = entrenar_w2v(general_sentences + condition_sentences)

In [ ]:
# 🔍 Función para ver palabras cercanas a una frase
def palabras_mas_cercanas_a_frase(frase, modelo, topn=10):
    palabras = limpiar_frase(frase).split()
    vectores = [modelo.wv[word] for word in palabras if word in modelo.wv]
    if not vectores:
        print("Ninguna palabra de la frase está en el vocabulario.")
        return []
    promedio = np.mean(vectores, axis=0)
    return modelo.wv.similar_by_vector(promedio, topn=topn)

print("🔹 Modelo BASE:")
for palabra, sim in palabras_mas_cercanas_a_frase(frase_input, w2v_base):
    print(f"{palabra}: {sim:.4f}")

print("\n🔸 Modelo COMBINADO (base + condiciones):")
for palabra, sim in palabras_mas_cercanas_a_frase(frase_input, w2v_combined):
    print(f"{palabra}: {sim:.4f}")

🔹 Modelo BASE:
correctamente.: 0.6662
otro: 0.6572
cualquier: 0.6542
balancea: 0.6490
terminado: 0.6413
ha: 0.6410
metal.: 0.6347
indica: 0.6315
lados: 0.6305
frías.: 0.6281

🔸 Modelo COMBINADO (base + condiciones):
alejado: 0.6720
respecto: 0.6658
descentrado: 0.6624
metálicos.: 0.6567
exacto: 0.6546
cuchara.: 0.6538
inclinando: 0.6511
afilado: 0.6435
balancea: 0.6418
lejos: 0.6371


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

def similitud_con_palabras_objetivo(frase, modelo, palabras_objetivo, topn=10):
    # Obtener palabras más cercanas al promedio de la frase
    palabras_modelo = palabras_mas_cercanas_a_frase(frase, modelo, topn=topn)
    palabras_encontradas = [palabra for palabra, _ in palabras_modelo if palabra in modelo.wv]

    if not palabras_encontradas:
        return np.nan  # or any other appropriate value
    vec_encontrado = np.mean([modelo.wv[p] for p in palabras_encontradas], axis=0)

    # Calcular promedio de palabras objetivo
    palabras_obj = [p for p in palabras_objetivo if p in modelo.wv]
    # Check if palabras_obj is empty
    if not palabras_obj:
        return np.nan  # or any other appropriate value
    vec_objetivo = np.mean([modelo.wv[p] for p in palabras_obj], axis=0)

    # Reshape the vectors to 2D if they are 1D
    vec_encontrado = vec_encontrado.reshape(1, -1)
    vec_objetivo = vec_objetivo.reshape(1, -1)

    # Cosine similarity
    sim = cosine_similarity(vec_encontrado, vec_objetivo)[0][0]
    return sim

In [ ]:
def check_words_in_vocab(words, model):
    in_vocab = {}
    for word in words:
        in_vocab[word] = word in model.wv
    return in_vocab

# Example usage:
vocab_check_base = check_words_in_vocab(palabras_objetivo, w2v_base)
print("Palabras objetivo en vocabulario base:")
print(vocab_check_base)

vocab_check_combined = check_words_in_vocab(palabras_objetivo, w2v_combined)
print("\nPalabras objetivo en vocabulario combinado:")
vocab_check_combined

Palabras objetivo en vocabulario base:
{'caida': False, 'ruptura': True, 'resbalar': False, 'tritura': False, 'impacto': True, 'desorden': True, 'riesgo': True, 'quebrado': True, 'roto': True}

Palabras objetivo en vocabulario combinado:


{'caida': False,
 'ruptura': True,
 'resbalar': True,
 'tritura': False,
 'impacto': True,
 'desorden': True,
 'riesgo': True,
 'quebrado': True,
 'roto': True}

In [ ]:
for frase_input in frases:
  sim_base = similitud_con_palabras_objetivo(frase_input, w2v_base, palabras_objetivo)
  sim_combinado = similitud_con_palabras_objetivo(frase_input, w2v_combined, palabras_objetivo)
  data_experimento = [archivo_base, archivo_condiciones, sim_base, sim_combinado]
  timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
  data_experimento.append(timestamp)
  new_row = pd.DataFrame([data_experimento], columns=['archivo_base', 'archivo_condiciones', 'sim_base', 'sim_combinado', 'timestamp'])
  lab_data = pd.concat([lab_data, new_row], ignore_index=True)

lab_data

,archivo_base,archivo_condiciones,sim_base,sim_combinado,timestamp
0,db__ventas_situaciones.json,db__ventas_etica.json,0.785407,0.804820,2025-04-17 20:18:46
1,db__ventas_situaciones.json,db__ventas_etica.json,0.639894,0.643459,2025-04-17 20:18:46
2,db__ventas_situaciones.json,db__ventas_etica.json,0.658628,0.719100,2025-04-17 20:18:46
3,db__general_phrases_v4.json,db__spatial_phrases.json,0.646315,0.816527,2025-04-17 20:19:35
4,db__general_phrases_v4.json,db__spatial_phrases.json,0.535361,0.568084,2025-04-17 20:19:35
5,db__general_phrases_v4.json,db__spatial_phrases.json,0.715575,0.648911,2025-04-17 20:19:35


In [ ]:
lab_data.to_excel(path+"results_v2.xlsx")